In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [61]:
# 这里使用的ID为1~N版本
class Person:
  def __init__(self, id):
    self.id = id
    self.alive = True # 是否存活
    self.offers = []  # 收到的offer
    self.job = -1 # 签约的企业

class Recruit:
  def __init__(self, id, num):
    self.id = id
    self.num = num # 岗位剩余容量
    # self.offers = []  # 收到的offer
    self.workers = [] # 签约的员工
    
class Pair:
  def __init__(self, per, rec, val):
    self.pid = per.id
    self.rid = rec.id
    self.val = val
    
  def __cmp__(self, another) -> bool:
    return self.val > another.val
  
  def print(self):
    print(self.pid, self.rid, self.val)

In [4]:
def get_ppd(per_id, job_id):
  return np.random.random()

def get_myd(per_id, job_id):
  return np.random.random()

def hardcore(per_id, job_id):
  return True

In [6]:
df_per = pd.read_excel('./user.xlsx')
df_per.head(1)

,序号,求职者 ID,姓名,预期岗位,工作地区,经验,性别,生日,政治面貌,自我评价,...,工作性质,简历关键词,工作经历,项目经历,竞赛经历,教育经历,培训经历,证书,语言,技能
0,1,1648621861036219904,张女士,"[""数据分析师""]","[""广东省"",""广州市"",""天河区""]",无经验,女,2002-12-18T16:00:00.000Z,LEAGUE_MEMBER,本科毕业，具有多年的比赛经验，曾获数据挖掘一等奖，数据分析一等奖，数学竞赛二等奖，数学建模一...,...,实习,[],[],[],[],[],[],[],[],[]


In [20]:
df_rec = pd.read_excel('./position.xlsx')
df_rec.head(1)

,序号,招聘信息 ID,企业名称,招聘岗位,学历要求,经验要求,岗位关键词,性质,岗位需求量,最大薪资,最小薪资,企业工作地点,公司类型,行业,公司规模,职位描述,职位福利
0,1,1648527394191049984,众云网,售前技术支持,大专,不限,"[{'id': '1648528107189178368', 'createTime': '...",全职,6,20000,12000,深圳市南山区粤海街道滨海社区海天一路19、17、18号软件产业基地4栋511,民营公司,"['互联网', '计算机软件']",150-500人,岗位职责：\n1、负责配合客户及项目需要，完成技术交流，提供解决方案；\n2、负责网络安全、...,"['餐饮补贴', '专业培训', '弹性工作']"


In [69]:
persons = []
recruits = []

In [70]:
for i in tqdm(range(df_per.shape[0])):
  id = df_per.loc[i, '序号']
  tmp = Person(id)
  persons.append(tmp)
  # print(id)

100%|██████████| 10877/10877 [00:00<00:00, 136025.46it/s]


In [71]:
for i in tqdm(range(df_rec.shape[0])):
  id = df_rec.loc[i, '序号']
  num = df_rec.loc[i, '岗位需求量']
  tmp = Recruit(id, num)
  recruits.append(tmp)
  # print(id)

100%|██████████| 1575/1575 [00:00<00:00, 13687.19it/s]


In [53]:
# sum = 0
# for i in recruits:
#   sum += i.num
# print(sum)
len(tmp_list)

6

In [74]:
epoch = 1
while True:
  # 招聘回合
  print(f'epoch {epoch}:')
  epoch += 1
  for rec in tqdm(recruits):  # 对于每个岗位
    # 发放offer
    tmp_list = [] # 临时数组：<per, rec, val>
    # 计算与每个求职者的匹配度
    for per in persons:
      val = get_ppd(per.id, rec.id)
      pair = Pair(per, rec, val)
      tmp_list.append(pair)
    # 就地排序 by val
    tmp_list.sort(reverse=True, key=lambda x:x.val)
    tmp_list = tmp_list[:rec.num] # 取TopK
    
    for pair in tmp_list:
      per = persons[pair.pid-1]
      rec = recruits[pair.rid-1]
      
      per.offers.append(rec.id) # 发放offer
  
  # 所有岗位offer发放完毕
    
  # 求职者签约
    for per in persons: # 对于每个求职者
      tmp2list = []
      for rid in per.offers:  # 对于每一份offer
        rec = recruits[rid-1]
        val = get_myd(per.id, rec.id)
        pair = Pair(per, rec, val)
        tmp2list.append(pair)
      if len(tmp2list)==0:  # 当前求职者无offer
        continue
      tmp2list.sort(reverse=True, key=lambda x:x.val) # 取最高满意度
      dest_rec = recruits[tmp2list[0].rid-1]
      if dest_rec.num > 0: # 若未招满
        dest_rec.num -= 1
        per.job = dest_rec.id
        dest_rec.worker.append(per)
     
      
      
      
  
  break
    
    

epoch 1:


  0%|          | 0/1575 [00:00<?, ?it/s]


AttributeError: 'Recruit' object has no attribute 'worker'

In [ ]:
for item in tmp_list:
  # print(item.pid, item.rid, item.val)
  item.print()

In [23]:
myd = np.ndarray((30, 10), dtype='float')
ppd = np.ndarray((30, 10), dtype='float')
for i in range(myd.shape[0]):
  for j in range(myd.shape[1]):
    myd[i][j] = get_myd(i,j)
    ppd[i][j] = get_ppd(i,j)
    # print(f'mark({i},{j}={mark[i][j]})')
myd.shape, ppd.shape

((30, 10), (30, 10))